<a href="https://colab.research.google.com/github/pulkit48/Explanation-using-DPO/blob/main/Score_Generation_for_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install evaluate bert_score rouge_score meteor

In [42]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/final_df_all_score.csv',index_col=False)
# temp=df['main_news']
# df['score6']=[None]*df.shape[0]
# temp_list_=[]
# for i in range(df.shape[0]):
#     temp_list_.append(df['main_news'][i][:-1].replace('[\n    Main news:',''))
# df['main_news']=temp_list_

In [50]:
df.head()

,News_description,main_news,explain_gpt,explain_gemini,index,explain_mistral_updated,News_description_updated,result,score1,score2,score3
0,लोकसभा चुनाव 2024 के लिए चुनाव आयोग द्वारा घोष...,प्रधानमंत्री नरेंद्र मोदी ने आंध्र प्रदेश के ...,इस समाचार को देखकर यह कहना उचित होगा कि यह **व...,यह समाचार वास्तविक प्रतीत होता है। इसमें प्रध...,0,यह समाचार **झूठा** है क्योंकि इसमें कई व्याकरण...,यह खबर पूरी तरह से सत्य है। लोकसभा चुनाव 2024 ...,सच,0.903637,1.020494,0.727220
1,प्रधानमंत्री नरेंद्र मोदी को लेकर अब कर्नाटक क...,कर्नाटक के मंत्री और कांग्रेस नेता शिवराज एस ...,"यह समाचार लेख असत्य प्रतीत होता है। पहले, इस त...",यह खबर असली लगती है। इसमें कर्नाटक के एक मंत्...,1,The article's authenticity is in question. It ...,प्रधानमंत्री नरेंद्र मोदी को लेकर कर्नाटक के म...,सच,1.166978,1.026627,0.616277
2,कांग्रेस नेता राहुल गांधी का एक वीडियो सोशल मी...,कांग्रेस नेता राहुल गांधी का एक वीडियो में उन...,इस समाचार को में **वास्तविक** मानता हूँ। इसकी ...,यह समाचार वास्तविक लगता है। लेख में राहुल गां...,2,यह समाचार लेख संभवतः फर्जी है क्योंकि यह सरल औ...,कांग्रेस नेता राहुल गांधी का एक वीडियो सोशल मी...,झूठ,0.936930,0.863038,0.749380
3,यूपी में हुए राज्यसभा चुनाव में भाजपा ने एक ती...,यूपी में राज्यसभा चुनाव के दौरान भाजपा ने समा...,यह समाचार **वास्तविक** प्रतीत होता है। इसकी पु...,यह समाचार लेख वास्तविक लगता है। इसमें राज्यसभ...,3,यह समाचार लेख भारत के उत्तर हिस्से में मौसम मे...,यूपी में हुए राज्यसभा चुनाव में भाजपा ने एक ती...,सच,1.041014,0.852126,0.809640
4,तृणमूल कांग्रेस नेता और निष्कासित लोकसभा सांसद...,तृणमूल कांग्रेस नेता महुआ मोइत्रा को प्रवर्तन...,यह समाचार पत्रिका का शीर्षक और विषय विशेष रूप ...,"दी गयी खबर की प्रकृति को देखते हुए, यह संभवतः ...",4,इस समाचार लेख में 2022 में गुजरात विधान सभा चु...,तृणमूल कांग्रेस नेता और निष्कासित लोकसभा सांसद...,सच,1.267304,1.111927,0.885214


In [ ]:
from evaluate import load
import numpy as np
# Load the ROUGE metric
meteor = load("meteor")
rouge=load('rouge')
bertscore = load("bertscore")
temp=['explain_gpt','explain_gemini','explain_mistral_updated']
scores=['score1','score2','score3']
for j,score in zip(temp,scores):
  if score not in df.columns:
    df[score]=[None]*df.shape[0]
  for i in range(0, df.shape[0]):

      # Skip rows where the score is already calculated
      # if df.loc[i, 'score6'] is not None:
      #     continue

      # Extract the reference and prediction for the current row
      references = [df.loc[i, 'News_description']]
      predictions = [df.loc[i, j]]

      # Compute ROUGE scores
      # if(i%10==0):
      #   print(references)
      #   print(predictions)
      results1 = meteor.compute(predictions=predictions, references=references)
      results2=rouge.compute(predictions=predictions, references=references)
      results3=bertscore.compute(predictions=predictions, references=references, model_type="bert-base-multilingual-cased")

      # results
      # break
      # Extract the desired ROUGE score (e.g., ROUGE-L F1 score)
      # df.loc[i, 'score1'] += results['meteor']  # ROUGE-L F1 score as an example
      df.loc[i, score]=  results3['f1'][0]+3*(results1['meteor']+results2['rougeL'])
      # Print progress
      # print(results['rouge1'])
      if(i%10==0):
        print(i)
  # Note: Adjust 'rougeL_fmeasure' to the specific ROUGE score you want (e.g., rouge1, rouge2, rougeL).


In [48]:
df.head()

,News_description,main_news,explain_gpt,explain_gemini,index,explain_mistral_updated,News_description_updated,result,score1,score2,score3
0,लोकसभा चुनाव 2024 के लिए चुनाव आयोग द्वारा घोष...,प्रधानमंत्री नरेंद्र मोदी ने आंध्र प्रदेश के ...,इस समाचार को देखकर यह कहना उचित होगा कि यह **व...,यह समाचार वास्तविक प्रतीत होता है। इसमें प्रध...,0,यह समाचार **झूठा** है क्योंकि इसमें कई व्याकरण...,यह खबर पूरी तरह से सत्य है। लोकसभा चुनाव 2024 ...,सच,0.903637,1.020494,0.727220
1,प्रधानमंत्री नरेंद्र मोदी को लेकर अब कर्नाटक क...,कर्नाटक के मंत्री और कांग्रेस नेता शिवराज एस ...,"यह समाचार लेख असत्य प्रतीत होता है। पहले, इस त...",यह खबर असली लगती है। इसमें कर्नाटक के एक मंत्...,1,The article's authenticity is in question. It ...,प्रधानमंत्री नरेंद्र मोदी को लेकर कर्नाटक के म...,सच,1.166978,1.026627,0.616277
2,कांग्रेस नेता राहुल गांधी का एक वीडियो सोशल मी...,कांग्रेस नेता राहुल गांधी का एक वीडियो में उन...,इस समाचार को में **वास्तविक** मानता हूँ। इसकी ...,यह समाचार वास्तविक लगता है। लेख में राहुल गां...,2,यह समाचार लेख संभवतः फर्जी है क्योंकि यह सरल औ...,कांग्रेस नेता राहुल गांधी का एक वीडियो सोशल मी...,झूठ,0.936930,0.863038,0.749380
3,यूपी में हुए राज्यसभा चुनाव में भाजपा ने एक ती...,यूपी में राज्यसभा चुनाव के दौरान भाजपा ने समा...,यह समाचार **वास्तविक** प्रतीत होता है। इसकी पु...,यह समाचार लेख वास्तविक लगता है। इसमें राज्यसभ...,3,यह समाचार लेख भारत के उत्तर हिस्से में मौसम मे...,यूपी में हुए राज्यसभा चुनाव में भाजपा ने एक ती...,सच,1.041014,0.852126,0.809640
4,तृणमूल कांग्रेस नेता और निष्कासित लोकसभा सांसद...,तृणमूल कांग्रेस नेता महुआ मोइत्रा को प्रवर्तन...,यह समाचार पत्रिका का शीर्षक और विषय विशेष रूप ...,"दी गयी खबर की प्रकृति को देखते हुए, यह संभवतः ...",4,इस समाचार लेख में 2022 में गुजरात विधान सभा चु...,तृणमूल कांग्रेस नेता और निष्कासित लोकसभा सांसद...,सच,1.267304,1.111927,0.885214


In [11]:
df.tail()

,News_description,main_news,explain_gpt,explain_gemini,index,explain_mistral_updated,News_description_updated,result,score1,score2,score3,score4,score5,score6
6539,सोशल मीडिया पर आए दिन कई खबरें बहुत तेजी से वा...,भारतीय खाद्य सुरक्षा और मानक प्राधिकरण (FSSAI...,यह खबर **सच्ची** है क्योंकि भारत के खाद्य सुरक...,यह समाचार वास्तविक प्रतीत होता है। क्योंकि इस...,6539,The article about the Congress party in Odisha...,NaN,झूठ,0.694923,0.702585,0.618617,NaN,0.108108,0.100719
6540,: कोरोना महामारी के आने के बाद लोगों की मोबाइल...,सोशल मीडिया पर एक संदेश में कहा गया है कि मुद...,"इस समाचार के आधार पर, मैं इसे **फर्जी** मानता ...",यह समाचार संदेश संभवतः फर्ज़ी है। इसमें कई संक...,6540,मैं इस समाचार को **सत्य** मानता हूँ। यह समाचार...,NaN,झूठ,0.700752,0.686367,0.684336,NaN,0.315789,0.181818
6541,सुप्रीम कोर्ट के फैसले से पहले दशकों तक इस बात...,अयोध्या में राम मंदिर का निर्माण बाबरी मस्जिद...,इस समाचार को मैं **सत्य** मानता हूँ। हाल के वर...,यह खबर आंशिक रूप से असत्य या भ्रामक प्रतीत होत...,6541,The article is not specified clearly as fake o...,NaN,झूठ,0.690485,0.682911,0.608382,NaN,0.363636,0.0
6542,लोकसभा चुनाव (Lok Sabha Elections 2024) को लेक...,पीएम मोदी ने केरल के पथानामथिट्टा में एक रैली...,यह समाचार वास्तविक प्रतीत होता है। प्रधानमंत्र...,यह समाचार वास्तविक लगता है। इसमें प्रधानमंत्र...,6542,मैं अनुमान लगाता हूँ कि यह समाचार लेख **फर्जी*...,NaN,सच,0.675215,0.656977,0.623842,NaN,0.285714,0.0
6543,गुजरात चुनाव प्रचार के दौरान पीएम मोदी ने गांध...,गुजरात चुनाव प्रचार के दौरान प्रधानमंत्री नरे...,यह समाचार वस्तुतः वास्तविक है। प्रधानमंत्री नर...,यह समाचार वास्तविक प्रतीत होता है। प्रधानमंत्...,6543,यह लेख नकली है। इसमें तथ्यात्मक जानकारी में त्...,NaN,झूठ,0.660911,0.652404,0.598696,NaN,0.0,0.0


In [49]:
df.to_csv('/content/drive/MyDrive/final_df_all_score.csv',index=0)